# Para Google Colaboratory (Drive)

In [0]:
from google.colab import drive

drive.mount('/content/drive')

In [23]:
#Deiv ruta
!ls "drive/My Drive/ML-20182/Notebooks/Training"

train_path = '/content/drive/My Drive/2018-2/Aprendiza de máquina/ML-20182/Notebooks/Training'
#!unzip -uq "drive/My Drive/ML-20182/Dataset&Papers/Dataset/fruits.zip" -d "drive/My Drive/ML-20182/Notebooks/Unzipeado/"

ls: cannot access 'drive/My Drive/ML-20182/Notebooks/Training': Transport endpoint is not connected


In [4]:
#Jorge ruta
ls -1 | wc -l '/content/drive/My Drive/2018-2/Aprendiza de máquina/ML-20182/Notebooks/Training/'

train_path = '/content/drive/My Drive/2018-2/Aprendiza de máquina/ML-20182/Notebooks/Training'

'Apple Braeburn'/        Dates/                 Pear/
'Apple Golden 1'/        Granadilla/           'Pear Abate'/
'Apple Golden 2'/       'Grapefruit Pink'/     'Pear Monster'/
'Apple Golden 3'/       'Grapefruit White'/    'Pear Williams'/
'Apple Granny Smith'/   'Grape Pink'/           Pepino/
'Apple Red 1'/          'Grape White'/          Physalis/
'Apple Red 2'/          'Grape White 2'/       'Physalis with Husk'/
'Apple Red 3'/           Guava/                 Pineapple/
'Apple Red Delicious'/   Huckleberry/          'Pineapple Mini'/
'Apple Red Yellow'/      Kaki/                 'Pitahaya Red'/
 Apricot/                Kiwi/                  Plum/
 Avocado/                Kumquats/              Pomegranate/
'Avocado ripe'/          Lemon/                 Quince/
 Banana/                'Lemon Meyer'/          Rambutan/
'Banana Red'/            Limes/                 Raspberry/
'Cactus fruit'/          Lychee/                Salak/
'Cantaloupe 1'/          Mandarine/          

# Para Github (Local)

In [28]:
#path
!ls ../data/

train_path_str = '../data/Training'
test_path_str = '../data/Test'

LICENSE  readme.md  Test  test-multiple_fruits	Training


# Imágenes - Preprocesamiento

In [29]:
import math
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm_notebook
from pathlib import Path
%matplotlib inline

In [35]:
##Extrayendo las categorias

def extract_categories(path_str):
    path = Path(path_str)
    categories = []
    for index, e in enumerate(path.iterdir()):
        categories.append((index, e))
    return categories


In [43]:
##Extrayendo las categorias (test)

categories_train = extract_categories(train_path_str)
categories_test = extract_categories(test_path_str)

print('Clases en Train: ', len(categories_train))
print('Clases en Test: ', len(categories_test))

Clases en Train:  81
Clases en Test:  81


In [44]:
#Revisamos que está en orden

#train

categories_train

[(0, PosixPath('../data/Training/Apple Red Delicious')),
 (1, PosixPath('../data/Training/Mango')),
 (2, PosixPath('../data/Training/Huckleberry')),
 (3, PosixPath('../data/Training/Pear Williams')),
 (4, PosixPath('../data/Training/Salak')),
 (5, PosixPath('../data/Training/Apple Red 1')),
 (6, PosixPath('../data/Training/Grapefruit White')),
 (7, PosixPath('../data/Training/Apple Golden 3')),
 (8, PosixPath('../data/Training/Carambula')),
 (9, PosixPath('../data/Training/Quince')),
 (10, PosixPath('../data/Training/Peach')),
 (11, PosixPath('../data/Training/Kumquats')),
 (12, PosixPath('../data/Training/Grape White')),
 (13, PosixPath('../data/Training/Banana Red')),
 (14, PosixPath('../data/Training/Grapefruit Pink')),
 (15, PosixPath('../data/Training/Passion Fruit')),
 (16, PosixPath('../data/Training/Cherry 1')),
 (17, PosixPath('../data/Training/Lemon')),
 (18, PosixPath('../data/Training/Apple Golden 2')),
 (19, PosixPath('../data/Training/Apple Red 3')),
 (20, PosixPath('../d

In [45]:
#test
categories_test

[(0, PosixPath('../data/Test/Apple Red Delicious')),
 (1, PosixPath('../data/Test/Mango')),
 (2, PosixPath('../data/Test/Huckleberry')),
 (3, PosixPath('../data/Test/Pear Williams')),
 (4, PosixPath('../data/Test/Salak')),
 (5, PosixPath('../data/Test/Apple Red 1')),
 (6, PosixPath('../data/Test/Grapefruit White')),
 (7, PosixPath('../data/Test/Apple Golden 3')),
 (8, PosixPath('../data/Test/Carambula')),
 (9, PosixPath('../data/Test/Quince')),
 (10, PosixPath('../data/Test/Peach')),
 (11, PosixPath('../data/Test/Kumquats')),
 (12, PosixPath('../data/Test/Grape White')),
 (13, PosixPath('../data/Test/Banana Red')),
 (14, PosixPath('../data/Test/Grapefruit Pink')),
 (15, PosixPath('../data/Test/Passion Fruit')),
 (16, PosixPath('../data/Test/Cherry 1')),
 (17, PosixPath('../data/Test/Lemon')),
 (18, PosixPath('../data/Test/Apple Golden 2')),
 (19, PosixPath('../data/Test/Apple Red 3')),
 (20, PosixPath('../data/Test/Pepino')),
 (21, PosixPath('../data/Test/Tomato Cherry Red')),
 (22, Po

In [46]:
#Conteo de imágenes por Training y Testing

def count_images(categories):
    count = 0
    for i,path in categories:
        fruit_path = Path(path)
        for img_fruit in fruit_path.iterdir():
            count += 1
    return count


In [47]:
# Revisamos cantidad de imagenes para cada set  (deberían ser 41322 y 13877, por info del repo de GitHub)
print('La cantidad de imágenes para  el train es ', count_images(categories_train))
print('La cantidad de imágenes para  el test es ', count_images(categories_test))


La cantidad de imágenes para  el train es  41322
La cantidad de imágenes para  el test es  13877


In [55]:
# Tamaño del training set: 41322 imgs
train_size = count_images(categories_train)

# Tamaño del test set: 13877 imgs
test_size = count_images(categories_test)

# Primero inicializamos los arrays que vamos a usar
x_train = np.ndarray(shape=(train_size, img_size, img_size,3), dtype=np.float32)
y_train = np.zeros(shape=(train_size), dtype=np.int8)
x_val = np.ndarray(shape=(test_size, img_size, img_size,3), dtype=np.float32)
y_val = np.zeros(shape=(test_size), dtype=np.int8)

In [56]:
# Definimos una funcion para leer una imagen y hacer el preprocesamiento
from keras.applications.resnet50 import preprocess_input
img_size = 100

def read_img(path):
    x = Image.open(path)
    #x = x.resize((img_size, img_size))
    x = np.asarray(x, np.float32)
    return preprocess_input(x)

In [ ]:
# Cargamos el train set
global_index = 0
for i,path in categories:
    print(i,path)
    
    train_path = Path(path)
    imagenes = []
    for index, e in enumerate(train_path.iterdir()):
      #print(global_index)
      x_train[global_index] = read_img(e)
      y_train[global_index] = i
      global_index += 1
print(len(x_train), len(y_train))

0 ../data/Training/Apple Red Delicious
1 ../data/Training/Mango
2 ../data/Training/Huckleberry
3 ../data/Training/Pear Williams
4 ../data/Training/Salak
5 ../data/Training/Apple Red 1
6 ../data/Training/Grapefruit White
7 ../data/Training/Apple Golden 3
8 ../data/Training/Carambula
9 ../data/Training/Quince
10 ../data/Training/Peach
11 ../data/Training/Kumquats
12 ../data/Training/Grape White
13 ../data/Training/Banana Red
14 ../data/Training/Grapefruit Pink
15 ../data/Training/Passion Fruit
16 ../data/Training/Cherry 1
17 ../data/Training/Lemon
18 ../data/Training/Apple Golden 2
19 ../data/Training/Apple Red 3
20 ../data/Training/Pepino
21 ../data/Training/Tomato Cherry Red
22 ../data/Training/Limes
23 ../data/Training/Cocos
24 ../data/Training/Rambutan
25 ../data/Training/Apple Red 2
26 ../data/Training/Lemon Meyer
27 ../data/Training/Nectarine
28 ../data/Training/Clementine
29 ../data/Training/Pineapple Mini
30 ../data/Training/Pineapple
31 ../data/Training/Banana
32 ../data/Trainin

In [42]:
# Cargamos el validation set
for i,e in tqdm_notebook(enumerate(cat_imgs[1000:1100] + dog_imgs[1000:1100])):
    x_val[i] = read_img(e)

y_val[100:] = 1 # cat -> 0 | dog -> 1

NameError: name 'cat_imgs' is not defined

# Cargamos una red entrenada
Vamos a cargar la red **ResNet50** ya entrenada, pero sin incluir las capas densas, ya que vamos a adaptar la red a nuestro caso específico.

In [ ]:
from keras.applications.resnet50 import ResNet50
base_model = ResNet50(include_top=False, input_shape=(img_size,img_size,3), pooling='avg')
base_model.summary()

In [ ]:
base_model.input, base_model.output

In [ ]:
# Dado que solo queremos entrenar las capas densas del modelo que agregaremos
# en el siguiente paso, vamos a setear "trainable = False" para que los pesos
# de la red entrenada no cambien.
base_model.trainable = False

# Creamos el modelo clasificador

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

lr = 0.01

top_model = Sequential([
    Dense(128, activation='relu', input_shape=(2048,)),
    Dense(1, activation='softmax')
])

top_model.compile(SGD(lr, momentum=0.9), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
top_model.summary()

# Juntamos los 2 modelos

In [ ]:
final_model = Sequential([base_model, top_model])

final_model.compile(SGD(lr, momentum=0.9),loss='sparse_categorical_crossentropy', metrics=['accuracy'])
final_model.summary()

# Entrenando en nuestro dataset

In [ ]:
# log = final_model.fit(x_train, y_train, batch_size=64, validation_data=[x_val, y_val])

La red de base que estamos usando (ResNet50) tiene bastantes capas y hacer todas estas operaciones toma un tiempo considerable, en CPU hacer este entrenamiento puede resultar impractico.
# Precompute
Si tenemos en cuenta que vamos a entrenar nuestro dataset un cierto número de épocas, en cada época se van a repetir exactamente las mismas operaciones en la misma data. Para no redundar, es bastante útil hacer un precompute de la data:
1. Pasar todas nuestras imagenes por la red base (ResNet50).
2. Guardamos los features extraidos.
3. Entrenamos las capas densas con los features extraidos.

In [ ]:
precomputed_train = base_model.predict(x_train, batch_size=128, verbose=1)
precomputed_train.shape

In [ ]:
precomputed_val = base_model.predict(x_val, batch_size=128, verbose=1)
precomputed_val.shape

# Entrenar a partir de los features extraidos

In [ ]:
# Ahora podemos usar un batch_size mas grande, ya que los features son mas pequeños
# que las imagenes.
log = top_model.fit(precomputed_train, y_train, epochs=5, batch_size=256, validation_data=[precomputed_val, y_val])

In [ ]:
def show_results(log):
    fig, axes = plt.subplots(1, 2, figsize=(14,4))
    ax1, ax2 = axes
    ax1.plot(log.history['loss'], label='train')
    ax1.plot(log.history['val_loss'], label='validation')
    ax1.set_xlabel('epoch'); ax1.set_ylabel('loss')
    ax2.plot(log.history['acc'], label='train')
    ax2.plot(log.history['val_acc'], label='validation')
    ax2.set_xlabel('epoch'); ax2.set_ylabel('accuracy')
    for ax in axes: ax.legend()

In [ ]:
show_results(log)

# Ejercicio: usando el modelo completo en el test set

In [ ]:
test_path = Path('../input/test/')
test_files = list(test_path.iterdir())

In [ ]:
def get_class(path):
    # Cargar la imagen del path
    img = Image.open(path)
    
    # Cambiar el tamaño de la imagen
    img_resized = img.resize((224, 224))
    
    # Cambiar a formato numpy y preprocesar
    x = np.asarray(img_resized, np.float32)[None]
    x = preprocess_input(x)
    
    # Obtener predicciones
    y = final_model.predict(x)
    
    # Decodear predicciones
    pred = 'cat' if y < 0.5 else 'dog'
    
    # Mostrar la imagen
    plt.imshow(img)
    plt.axis('off')
    plt.title(pred, size=14)
    
    return

In [ ]:
sample = np.random.choice(test_files)
get_class(sample)